# <center>NumPy案例</center>

### 实践题目
#### 泰坦尼克乘客数据分析生还率
* 著名的数据分析竞赛网站Kaggle上，举行了很多数据分析比赛，其中比较著名的就有 泰坦尼克号乘客生还预测 。
* Kaggle提供的数据集中，共有1309名乘客数据，其中891是已知存活情况，剩下418则是需要进行分析预测的。

#### 我们想知道
1. 生还者总数
2. 平均生还率

#### 进一步
3. 各等级船舱乘客数量
4. 各等级船舱生还数量
5. 不同舱位生还率

#### 更进一步
6. 不同性别平均生还率 ['Sex','Survived']

### 数据文件
* titanic-data.csv

### 本节任务
* 分析探索原始数据，查看数据空值等情况
* 构建特征工程，统计特征分布与生还关系

### 数据主要字段说明如下：
* PassengerId: 乘客编号
* Survived: Survived (1) or died (0)是否存活
* Pclass: 船舱
* Name: 姓名
* Sex: 性别
* Age: 年龄
* SibSp: 兄弟/姐妹/配偶的数量
* Parch: 父母/子女的数量
* Ticket: 票号
* Fare: 票价
* Cabin: 座号
* Embarked: 登船港口

In [59]:
import os
import numpy as np

In [60]:
#获取文件的当前路径
current_dir = os.path.dirname(os.path.realpath('__file__'))
current_dir

'C:\\Users\\wzqcg\\PythonCode'

In [61]:
#设置读取文件的路径
filename = os.path.join(current_dir, 'titanic-data.csv')
filename

'C:\\Users\\wzqcg\\PythonCode\\titanic-data.csv'

In [62]:
#在这里直接读取首行的时候会出现换行符 \n 所以利用切边给它过滤掉，在通过字符串分割转换成列表
with open(filename,encoding='utf-8') as f:
    head_index=np.array(f.readline()[:-1].split(','))
head_index

array(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='<U11')

In [63]:
data = np.loadtxt('titanic-data.csv', delimiter=',',dtype=np.str, skiprows=1) #从文件中加载数据直接生成array，跳过第一行
data

array([['1', '0', '3', ..., '7.25', '', 'S'],
       ['2', '1', '1', ..., '71.2833', 'C85', 'C'],
       ['3', '1', '3', ..., '7.925', '', 'S'],
       ...,
       ['889', '0', '3', ..., '23.45', '', 'S'],
       ['890', '1', '1', ..., '30', 'C148', 'C'],
       ['891', '0', '3', ..., '7.75', '', 'Q']], dtype='<U83')

In [64]:
# 查看首行数据
first_line = data[0]
first_line

array(['1', '0', '3', '"Braund Mr. Owen Harris"', 'male', '22', '1', '0',
       'A/5 21171', '7.25', '', 'S'], dtype='<U83')

In [65]:
 # 有效字段索引
a = np.arange(12)
data_index = np.delete(a, 3) # name字段的值对生还率预测没有意义，可以删除
data_index


array([ 0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11])

In [66]:
 # 去除name索引数据，通过整数数组来取数据
head_index = head_index[data_index]
head_index


array(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'], dtype='<U11')

In [67]:
 # 去除name那一列数据
corpus_data = data[:,data_index] #数组切片
corpus_data


array([['1', '0', '3', ..., '7.25', '', 'S'],
       ['2', '1', '1', ..., '71.2833', 'C85', 'C'],
       ['3', '1', '3', ..., '7.925', '', 'S'],
       ...,
       ['889', '0', '3', ..., '23.45', '', 'S'],
       ['890', '1', '1', ..., '30', 'C148', 'C'],
       ['891', '0', '3', ..., '7.75', '', 'Q']], dtype='<U83')

### 缺失值分析
* Age和Cabin字段有较多缺失值，Embanked字段有2个缺失值

In [68]:
items_data = corpus_data.T #矩阵转置， 旋转90度，原来每一列 成为 新矩阵的每一行
items_data.shape #items_data有11行

(11, 891)

In [69]:
null_data = []
for col in items_data: #对于 items_data每一行 复制给变量col
    null_data.append(sum(col=='')) #计算这一行空值个数
null_data


[0, 0, 0, 0, 177, 0, 0, 0, 0, 687, 2]

### 处理缺失值
#### 需要进行缺失值处理的有：Age、Cabin、Embarked

#### Age填充“平均值”

In [70]:
 # age属性中有缺失, 通过计算该属性的均值将缺失处填补,使得数据的数量一致
age_index = np.argwhere(head_index == 'Age') #查找索引
print(age_index.dtype)
print("age_index:%d"%age_index) 
age_data = items_data[4]
age_data_null = sum(age_data=='')
handle_data = np.where(age_data == '', 0, age_data)#age_data == '' Ture时都换成0，False时换成age_data
age_mean = sum(handle_data.astype(np.float))/(len(handle_data)-age_data_null) #非空的年龄的平均值
print(len(handle_data))
age_mean

int64
age_index:4
891


29.69911764705882

In [71]:
items_data[4] = np.where(items_data[4] == '', age_mean, items_data[4]) #数组赋值为数组,用均值替换0值

#### Embarked填充“众数”

In [72]:
#embarked填充众数
embarked_index = 10
embarked_data = items_data[embarked_index ]


In [73]:
np.unique(embarked_data) #返回有几种值

array(['', 'C', 'Q', 'S'], dtype='<U83')

In [74]:
from collections import Counter
Counter(embarked_data) #统计不同舱位数量

Counter({'S': 644, 'C': 168, 'Q': 77, '': 2})

In [75]:
items_data[10] = np.where(items_data[10] == '', 'S', items_data[10]) #填充缺失值为 众

#### Cabin 删除

In [76]:
head_index = np.delete(head_index,9)
items_data = np.delete(items_data,9, axis = 0)
print(head_index)
print(items_data)

['PassengerId' 'Survived' 'Pclass' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket'
 'Fare' 'Embarked']
[['1' '2' '3' ... '889' '890' '891']
 ['0' '1' '1' ... '0' '1' '0']
 ['3' '1' '3' ... '3' '1' '3']
 ...
 ['A/5 21171' 'PC 17599' 'STON/O2. 3101282' ... 'W./C. 6607' '111369'
  '370376']
 ['7.25' '71.2833' '7.925' ... '23.45' '30' '7.75']
 ['S' 'C' 'S' ... 'S' 'C' 'Q']]


### 数据离散化

In [78]:
# 获取sex的值, 并用0和1代表男性和女性
np.unique(items_data[3])

array(['female', 'male'], dtype='<U83')

In [79]:
items_data[3] = np.where(items_data[3] == 'male', 0, items_data[3])
items_data[3] = np.where(items_data[3] == 'female', 1, items_data[3])

In [80]:
# 获取embarked的值, 用0,1,2分别表示S,C,Q
np.unique(items_data[9])
items_data[9] = np.where(items_data[9] == 'S', 0, items_data[9])
items_data[9] = np.where(items_data[9] == 'C', 1, items_data[9])
items_data[9] = np.where(items_data[9] == 'Q', 2, items_data[9])


### 数据分析1

In [82]:
# 生还者总数
items_data[1].astype(np.int32).sum(axis=0) #一维数组 axis只有0，可以不写


342

In [85]:
# 平均生还率
items_data[1].astype(np.int32).mean(axis=0)

0.3838383838383838

### 数据分析2

In [89]:
# 各等级船舱Pclass乘客总数量统计：
from collections import Counter
data = np.array(items_data[2])
Counter(data)

Counter({'3': 491, '1': 216, '2': 184})

In [101]:
# 各等级船舱生还数量统计：'Pclass', 'Survived'
pclass_survived = np.vstack((items_data[2],items_data[1]))
pclass_survived
unique, counts = np.unique(pclass_survived, return_counts=True,axis=1) #col有几种，各有多少

array([['3', '1', '3', ..., '3', '1', '3'],
       ['0', '1', '1', ..., '0', '1', '0']], dtype='<U83')

In [102]:
unique

array([['1', '1', '2', '2', '3', '3'],
       ['0', '1', '0', '1', '0', '1']], dtype='<U83')

In [103]:
counts

array([ 80, 136,  97,  87, 372, 119], dtype=int64)

- 1 0 组合有80人；1表示头等舱，0表示失踪
- 1 1 组合有136人；

- 2 0 组合有97人；
- 2 1 组合有87人；

- 3 0 组合有97人；
- 2 1 组合有87人；

In [104]:
a = [216, 184, 491]
b = [136, 87, 119]
np.array(b)/np.array(a) #不同舱位生还率


array([0.62962963, 0.47282609, 0.24236253])

### 数据分析3
#### 不同性别平均生还率 ['Sex','Survived']

In [108]:
sex_index = np.argwhere(head_index == 'Sex')[0][0]
sex_index


3

In [110]:
#不同性别平均生还率 ['Sex','Survived']
sex_survived = np.vstack((items_data[3],items_data[1]))
unique, counts = np.unique(sex_survived, return_counts=True,axis=1)

In [113]:
unique

array([['0', '0', '1', '1'],
       ['0', '1', '0', '1']], dtype='<U83')

In [114]:
counts

array([468, 109,  81, 233], dtype=int64)

In [116]:
from collections import Counter
data = np.array(items_data[3])
Counter(data)


Counter({'0': 577, '1': 314})

In [118]:
a = np.array([577,314])
b = np.array([109,233])

In [119]:
b/a

array([0.18890815, 0.74203822])

#### 乘客中男性共577人，女性314人，女性生还率为74.2%，远高于男性的18.89%。